In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost
from sklearn.ensemble import RandomForestClassifier
import joblib
from joblib import load
import lime
import lime.lime_tabular
import shap

In [2]:
#I had to downgrade sklearn for joblib to load the dataset correctly

#pip uninstall sklearn --yes
#pip install -v scikit-learn==1.2.2

In [3]:
#loading in trained model
model = joblib.load('best_random_forest9.pkl')

In [4]:
#making sure the model loaded correctly
type(model)

sklearn.ensemble._forest.RandomForestClassifier

In [5]:
#load in the dataset
df = pd.read_csv("cleaned_data.csv")

# take random sample of dataframe to reduce size
df = df.sample(n=200000, random_state=42)

# Drop discharge variable which was not dropped during preprocessing

df = df.drop("YEAR_OF_DISCHARGE", axis=1)

#df.head()

In [6]:
#removing/filtering 'transferred to another treatment' option from data
df = df[df['REASON_FOR_DISCHARGE'] != 'Transferred to another treatment program or facility']

In [7]:
# Create target variable. If the patient completed treatment and had no prior treatment episodes,
#they are considered a success. Otherwise, they are considered a failure.

df['SUCCESSFUL_TREATMENT'] = df.apply(lambda row: 1 if row['REASON_FOR_DISCHARGE'] == 'Treatment completed' and row['PRIOR_TREATMENT_EPISODES'] == "No prior treatment episode" else 0, axis=1)

In [8]:
#creating train/test splits
target = df['SUCCESSFUL_TREATMENT']
features = df.drop(['REASON_FOR_DISCHARGE', 'PRIOR_TREATMENT_EPISODES', 'SUCCESSFUL_TREATMENT'], axis=1)
features_one_hot = pd.get_dummies(features)

X_train, X_test, y_train, y_test = train_test_split(features_one_hot, target, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(106172, 213) (26544, 213) (106172,) (26544,)


# LIME for Bias Analysis

In [9]:
#fitting model for LIME
clf = model.fit(X_train, y_train)

In [10]:
# Create an explainer object
explainer = lime.lime_tabular.LimeTabularExplainer(X_train, feature_names=X_train.columns, discretize_continuous=False)

# Explain the predictions for a specific instance
i = 213
exp = explainer.explain_instance(X_test.iloc[i], clf.predict_proba, num_features=213)

# Print the explanation
print(exp.as_list())

X does not have valid feature names, but RandomForestClassifier was fitted with feature names


[('PRIMARY_SUBSTANCE_ABUSE_Heroin', -0.01019573014535909), ('DSM_DIAGNOSIS_Opioid dependence', -0.008511370348024357), ('PRIMARY_PAYMENT_METHOD_Medicaid', -0.00842278886457436), ('FREQUENCY_OF_SELF_HELP_ATTENDANCE_No attendance', 0.008344736587126901), ('PSYCHIATRIC_PROBLEM_Yes', -0.0074415783259588475), ('ALCOHOL_OR_DRUG_ABUSE_Alcohol only', 0.007293555453512636), ('PSYCHIATRIC_PROBLEM_No', 0.006572043204190805), ('EMPLOYMENT_AT_DISCHARGE_Full time', 0.005603882902243389), ('HEALTH_INSURANCE_Medicaid', -0.004745710221429763), ('HEALTH_INSURANCE_Private insurance, Blue Cross/Blue Shield, HMO', 0.004333749623918976), ('SERVICES_AT_DISCHARGE_Ambulatory, intensive outpatient', -0.0037801944369011023), ('STATE_Kentucky', 0.0037077928422981667), ('LIVING_ARRANGEMENT_AT_DISCHARGE_Homeless', -0.0037058461696681992), ('PRIMARY_SUBSTANCE_ABUSE_Marijuana/hashish', 0.003270420491915302), ('LIVING_ARRANGEMENT_AT_DISCHARGE_Independent living', 0.0027247048578713197), ('ALCOHOL_OR_DRUG_ABUSE_Alcohol

In [11]:
#creating list of lime explainer output
exp_list = exp.as_list()

#sorting, rounding, and filtering the list of tuples via the RACE variables
exp_list_rounded = [(item[0],round(item[1],10)) for item in exp_list]
exp_list_sorted = sorted(exp_list_rounded, key=lambda x: x[1])
exp_list_filtered_race = [t for t in exp_list_sorted if t[0].startswith('RACE_')]
exp_list_filtered_race

[('RACE_Other single race', -0.0002364125),
 ('RACE_Alaskan Native', -0.0002228478),
 ('RACE_Black or African American', -0.0001637954),
 ('RACE_American Indian', -0.0001145825),
 ('RACE_Asian', 3.14651e-05),
 ('RACE_Two or more races', 7.7077e-05),
 ('RACE_Native Hawaiian or Other Pacific Islander', 0.0001068427),
 ('RACE_White', 0.0002616578)]

# SHAP for Bias Analysis

In [12]:
#setting up SHAP for Bias Analysis

# Train an xgboost model
X= features_one_hot
y = df['SUCCESSFUL_TREATMENT']

#creating model
clf = xgboost.XGBClassifier()

#fitting model
clf.fit(X, y)

# Create an explainer object
explainer = shap.Explainer(clf, X, feature_names=X.columns)

# Explain the predictions for a specific instance
i = len(X) - 1
exp = explainer.shap_values(X.iloc[i])

# Print the explanation
print(exp)

[-1.53666479e-02  0.00000000e+00 -5.29806206e-03 -1.06839821e-02
 -7.83845947e-03 -6.65100535e-03  6.54167079e-03  4.03566028e-03
 -9.36123673e-02  7.55841520e-03  8.53601436e-03  0.00000000e+00
  2.21370497e-02  0.00000000e+00 -6.47150890e-04  1.11100261e-02
  4.45844457e-04  2.08317540e-02  0.00000000e+00 -1.08866174e-03
 -1.89766868e-02  9.91438143e-03  1.95569221e-03  5.17569950e-03
  1.67657790e-01 -8.57641245e-05  6.33195784e-03 -1.07532221e-02
 -1.38236278e-03 -2.06902302e-03  3.97495315e-03  1.63299162e-04
 -4.73919060e-02  3.45611332e-02  3.07055300e-02 -7.44756837e-02
 -4.16843687e-02 -6.60296295e-03 -5.49267422e-03 -2.44708389e-03
  9.44381640e-03 -6.47640260e-03 -2.09090135e-02  4.07752877e-02
 -5.12384397e-02  4.15024534e-05 -4.11965684e-03  0.00000000e+00
 -1.93369003e-01 -2.44231671e-01 -1.34694902e-04 -4.99443807e-03
 -9.10478693e-04 -2.93689915e-01 -1.31198259e-01 -2.15601146e-03
  0.00000000e+00  2.35401982e-03 -2.93300887e-02  0.00000000e+00
  9.37256900e-02 -2.21130

In [13]:
#combining shap_values and feature names into a list of tuples
shap_values_list = list(exp)
feature_names_list = list(explainer.data_feature_names)
combined_shap_list = list(zip(feature_names_list, shap_values_list))

In [14]:
#sorting, rounding, and filtering the list of tuples via the RACE variables
combined_shap_list_rounded = [(item[0],round(item[1],10)) for item in combined_shap_list]
combined_shap_list_sorted = sorted(combined_shap_list_rounded, key=lambda x: x[1])
combined_shap_list_filtered_race = [t for t in combined_shap_list_sorted if t[0].startswith('RACE_')]
combined_shap_list_filtered_race

[('RACE_Two or more races', -0.0189766868),
 ('RACE_Other single race', -0.0010886617),
 ('RACE_Alaskan Native', -0.0006471509),
 ('RACE_Native Hawaiian or Other Pacific Islander', 0.0),
 ('RACE_Asian', 0.0004458445),
 ('RACE_White', 0.0099143814),
 ('RACE_American Indian', 0.0111100261),
 ('RACE_Black or African American', 0.020831754)]

# source : https://deepchecks.com/reducing-bias-and-ensuring-fairness-in-machine-learning/#